In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, RNN, Dense


In [ ]:

data = pd.read_csv("sent.csv")

data = data.dropna()
texts = data["clean_text"]
labels = data["category"]

processed_texts = []
for text in texts:
    if not isinstance(text, str):
        print("Non-string value encountered:", text)
        continue

    clean_text = BeautifulSoup(text, "html.parser").get_text()

    clean_text = re.sub(r"[^\w\s]", "", clean_text)  # Re import needed
    clean_text = re.sub(r"\d+", "", clean_text)

    tokens = nltk.word_tokenize(clean_text)

    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    processed_texts.append(" ".join(tokens))


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


NameError: name 're' is not defined

In [ ]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_texts)
vocab_size = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(processed_texts)

max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:

model = Sequential([
    Embedding(vocab_size, 100, input_length=max_length),  # Adjust embedding_dim
    LSTM(64),  # Or RNN(64)
    Dense(3, activation="softmax")  # 3 output units for 3 sentiment classes
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

from keras.utils import to_categorical


one_hot_labels = to_categorical(labels, num_classes=3)  # Assuming 3 classes (-1, 0, 1)

model.fit(padded_sequences, one_hot_labels, epochs=10, batch_size=32)



In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2)  # 20% for testing
loss, accuracy = model.evaluate(X_train, y_train)
print("Training Accuracy:", accuracy)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

# Generate predictions for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Analyze performance metrics
print("Confusion Matrix:\n", confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes))
print("Classification Report:\n", classification_report(np.argmax(y_test, axis=1), y_pred_classes))


4075/4075 [==============================] - 22s 5ms/step - loss: 0.0089 - accuracy: 0.9974
Training Accuracy: 0.9973844885826111
1019/1019 [==============================] - 5s 5ms/step - loss: 0.0089 - accuracy: 0.9975
Test Accuracy: 0.9974842071533203
1019/1019 [==============================] - 5s 5ms/step
Confusion Matrix:
 [[11098     9     4]
 [   34 14405     8]
 [   17    10  7009]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     11111
           1       1.00      1.00      1.00     14447
           2       1.00      1.00      1.00      7036

    accuracy                           1.00     32594
   macro avg       1.00      1.00      1.00     32594
weighted avg       1.00      1.00      1.00     32594



In [ ]:
import pickle

with open("sent.pkl", "wb") as f:
    pickle.dump(model, f)



In [ ]:
import re
import nltk

from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

def preprocess_and_predict(text):

    clean_text = BeautifulSoup(text, "html.parser").get_text()
    clean_text = re.sub(r"[^\w\s]", "", clean_text)
    clean_text = re.sub(r"\d+", "", clean_text)
    tokens = nltk.word_tokenize(clean_text)
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    preprocessed_text = " ".join(tokens)
    print(preprocessed_text)

    with open("sent.pkl", "rb") as f:
        model = pickle.load(f)

    # sequences = tokenizer.texts_to_sequences(preprocessed_text)
    # vectorized_text = tokenizer.texts_to_sequences(preprocessed_text)
    vectorized_text = tokenizer.texts_to_sequences([tokens])
    max_length = max([len(seq) for seq in vectorized_text])
    print(vectorized_text)

    vectorized_text = pad_sequences([vectorized_text], maxlen=43)
    vectorized_text = vectorized_text.reshape( -1, 43)

    # vectorized_text = vectorized_text.reshape(None, 43, 100)
    # tf.squeeze(vectorized_text, axis=0)

    print(vectorized_text.shape)

    sentiment_prediction = model.predict(vectorized_text)

    print("Preprocessed text:", preprocessed_text)
    final_prediction = sentiment_prediction[-1, :].argmax()
    print("Predicted sentiment:", final_prediction)
    if(final_prediction==0):
        print("neutral sentence")
    elif(final_prediction==1):
        print("good sentence")
    elif(final_prediction==2):
        print("bad sentence")

while True:
    user_input = input("Enter text (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        break
    preprocess_and_predict(user_input)


Enter text (or type 'exit' to quit):  good


good
[[34]]
(1, 43)
1/1 [==============================] - 1s 860ms/step
Preprocessed text: good
Predicted sentiment: 1
good sentence


Enter text (or type 'exit' to quit):  i hate you


hate
[[123]]
(1, 43)
1/1 [==============================] - 1s 956ms/step
Preprocessed text: hate
Predicted sentiment: 2
bad sentence


Enter text (or type 'exit' to quit):  i hate you but also love you


...

Enter text (or type 'exit' to quit):  a scientific research says all about sky


scientific research say sky
[[1505, 999, 14, 1369]]
(4, 43)
1/1 [==============================] - 1s 872ms/step
Preprocessed text: scientific research say sky
Predicted sentiment: 0
neutral sentence


Enter text (or type 'exit' to quit):  i am thirsty


thirsty
[[11372]]
(1, 43)
1/1 [==============================] - 1s 871ms/step
Preprocessed text: thirsty
Predicted sentiment: 0
neutral sentence


Enter text (or type 'exit' to quit):  exit


In [ ]:
model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 43, 100)           9654000   
                                                                 
 lstm_9 (LSTM)               (None, 64)                42240     
                                                                 
 dense_19 (Dense)            (None, 3)                 195       
                                                                 
Total params: 9696435 (36.99 MB)
Trainable params: 9696435 (36.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
